In [62]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir

In [63]:
def rot_x(theta):
    rot_x = sp.Matrix([[1,0,0],
                       [0,sp.cos(theta),-sp.sin(theta)],
                       [0,sp.sin(theta),sp.cos(theta)]])
    return rot_x

def rot_y(theta):
    rot_y = sp.Matrix([[sp.cos(theta),0,sp.sin(theta)],
                       [0,1,0],
                       [-sp.sin(theta),0,sp.cos(theta)]])
    return rot_y

def rot_z(theta):
    rot_z = sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],
                       [sp.sin(theta),sp.cos(theta),0],
                       [0,0,1]])
    return rot_z

def trans(a,b,c):
    x = sp.Matrix([[a],[b],[c]])
    return x

def get_torque_from_L(L,q,qd):
    round_L_round_q = sp.zeros(len(q),1);
    i = 0;
    for q_i in q:
        round_L_round_q_i = [];
        round_L_round_q_i = sp.diff(L,q_i);
        round_L_round_q[i] = round_L_round_q_i;
        i+=1;
     
    d_dt_round_L_round_qd = sp.zeros(len(qd),1);
    i = 0;
    for qd_i in qd:
        round_L_round_qd_i = [];
        d_dt_round_L_round_qd_i = [];
        round_L_round_qd_i = sp.diff(L,qd_i);
        d_dt_round_L_round_qd_i = time_derivative(round_L_round_qd_i,N);
        d_dt_round_L_round_qd[i] = d_dt_round_L_round_qd_i;
        i+=1;
        
    tau = d_dt_round_L_round_qd - round_L_round_q 
    return tau

In [64]:
r, L_1, l_t = sp.symbols('r, L_1, l_t')
m_w, m_b, m_1, g = sp.symbols('m_w, m_b, m_1, g')
I_w, I_b, I_1 = sp.symbols('I_w, I_b, I_1')
theta_P, theta_w = dynamicsymbols('theta_P, theta_w')

# Displacement Vector
x_c = trans(r*theta_w,0,0)
x_w = x_c +  trans(0,0,r)
x_1 = x_w +  rot_y(theta_P) @ trans(0,0,L_1)
x_b = x_w +  rot_y(theta_P) @ trans(0,0,l_t)


# Velocity Vector
v_w = sp.simplify(time_derivative(x_w,N))
v_b = sp.simplify(time_derivative(x_b,N))
v_1 = sp.simplify(time_derivative(x_1,N))

# Angular Velocity Vector
w_w = trans(0, theta_w.diff(), 0)
w_b = trans(0, theta_P.diff(), 0)
w_1 = trans(0, theta_P.diff(), 0)


# Generalized Coordinates
q = sp.Matrix([[theta_P], [theta_w]])
qd = q.diff()
qdd = qd.diff()

# Kinetic Energy
T_trans = 0.5*(m_w*v_w.dot(v_w) + m_b*v_b.dot(v_b) + m_1*v_1.dot(v_1))
#T_body = I_by * w_b[1]**2 
#T_wheel = I_wy * w_w[1]**2

T_body = I_b* w_b[1]**2 
T_wheel =I_w* w_w[1]**2 
T_1 =I_1* w_1[1]**2 
#T_motor = I_my * n**2 * (theta_w.diff() - theta_P.diff())**2
#T_gimbal_right = I_lfx * w_lf[0]**2 + I_lfy * w_lf[1]**2 + I_lfz * theta_gb.diff()**2
#T_gimbal_left = I_rfx * w_rf[0]**2 + I_rfy * w_rf[1]**2 + I_rfz * theta_gb.diff()**2
T_rot = 0.5*(T_body + T_wheel + T_1 )

#T_rot = 0.5*(I_wy*w_w[1]**2 + I_by*w_b[1]**2 + I_my*n*n*(theta_w.diff()**2) + I_lfz*w_lf[2]**2 + I_rfz*w_rf[2]**2 + I_lfy*w_lf[1]**2 + I_rfy*w_rf[1]**2)
#I_my*n*n*(theta_w.diff()-theta_P.diff())**2,  I_bx*(theta_R.diff()-theta_gb.diff())**2
# Potential Energy
V =  m_b*g*(l_t*sp.cos(theta_P)) + m_1*g*(L_1*sp.cos(theta_P)) 

# Lagrangian
L = T_trans + T_rot - V


In [65]:
tau = get_torque_from_L(L,q,qd)
tau = sp.simplify(tau)
# Ml, Cl, Gl, Wl = ir.get_EoM_from_T(tau,qdd,g)
# Ml, Cl, Gl, Wl

In [66]:
tau

Matrix([
[                       1.0*I_1*Derivative(theta_P(t), (t, 2)) + 1.0*I_b*Derivative(theta_P(t), (t, 2)) + 1.0*L_1**2*m_1*Derivative(theta_P(t), (t, 2)) - 1.0*L_1*g*m_1*sin(theta_P(t)) + 1.0*L_1*m_1*r*cos(theta_P(t))*Derivative(theta_w(t), (t, 2)) - 1.0*g*l_t*m_b*sin(theta_P(t)) + 1.0*l_t**2*m_b*Derivative(theta_P(t), (t, 2)) + 1.0*l_t*m_b*r*cos(theta_P(t))*Derivative(theta_w(t), (t, 2))],
[1.0*I_w*Derivative(theta_w(t), (t, 2)) + 1.0*m_1*r*(-L_1*sin(theta_P(t))*Derivative(theta_P(t), t)**2 + L_1*cos(theta_P(t))*Derivative(theta_P(t), (t, 2)) + r*Derivative(theta_w(t), (t, 2))) + 1.0*m_b*r*(-l_t*sin(theta_P(t))*Derivative(theta_P(t), t)**2 + l_t*cos(theta_P(t))*Derivative(theta_P(t), (t, 2)) + r*Derivative(theta_w(t), (t, 2))) + 1.0*m_w*r**2*Derivative(theta_w(t), (t, 2))]])

In [67]:
T_w, F_b = sp.symbols('T_w, F_b')
u = sp.Matrix([[T_w], [F_b]])

u_matrix = sp.Matrix([[-T_w - F_b*l_t*sp.cos(theta_P)], [T_w - F_b*l_t*sp.cos(theta_P)]])

In [68]:
tau_eq = tau+u_matrix

In [69]:
M, C, G, W = get_EoM_from_T(tau_eq,qdd,g, u)

In [70]:
M

Matrix([
[1.0*I_1 + 1.0*I_b + 1.0*L_1**2*m_1 + 1.0*l_t**2*m_b,            1.0*r*(L_1*m_1 + l_t*m_b)*cos(theta_P(t))],
[          1.0*r*(L_1*m_1 + l_t*m_b)*cos(theta_P(t)), 1.0*I_w + 1.0*m_1*r**2 + 1.0*m_b*r**2 + 1.0*m_w*r**2]])

In [71]:
C

Matrix([
[                                                                      0],
[-1.0*r*(L_1*m_1 + l_t*m_b)*sin(theta_P(t))*Derivative(theta_P(t), t)**2]])

In [72]:
G

Matrix([
[-1.0*g*(L_1*m_1 + l_t*m_b)*sin(theta_P(t))],
[                                         0]])

In [73]:
W

Matrix([
[ 1, l_t*cos(theta_P(t))],
[-1, l_t*cos(theta_P(t))]])

In [74]:
linearlize_eq = {sp.sin(theta_P):theta_P, sp.cos(theta_P):1, theta_P.diff()**2:0}
tau_linear = sp.simplify(tau_eq.subs(linearlize_eq))
tau_linear

Matrix([
[-1.0*F_b*l_t + 1.0*I_1*Derivative(theta_P(t), (t, 2)) + 1.0*I_b*Derivative(theta_P(t), (t, 2)) + 1.0*L_1**2*m_1*Derivative(theta_P(t), (t, 2)) - 1.0*L_1*g*m_1*theta_P(t) + 1.0*L_1*m_1*r*Derivative(theta_w(t), (t, 2)) - 1.0*T_w - 1.0*g*l_t*m_b*theta_P(t) + 1.0*l_t**2*m_b*Derivative(theta_P(t), (t, 2)) + 1.0*l_t*m_b*r*Derivative(theta_w(t), (t, 2))],
[                                                                               -F_b*l_t + 1.0*I_w*Derivative(theta_w(t), (t, 2)) + T_w + 1.0*m_1*r*(L_1*Derivative(theta_P(t), (t, 2)) + r*Derivative(theta_w(t), (t, 2))) + 1.0*m_b*r*(l_t*Derivative(theta_P(t), (t, 2)) + r*Derivative(theta_w(t), (t, 2))) + 1.0*m_w*r**2*Derivative(theta_w(t), (t, 2))]])

In [75]:
Ml, Cl, Gl, Wl = get_EoM_from_T(tau_linear,qdd,g, u)

In [76]:
Ml

Matrix([
[1.0*I_1 + 1.0*I_b + 1.0*L_1**2*m_1 + 1.0*l_t**2*m_b,                            1.0*r*(L_1*m_1 + l_t*m_b)],
[                          1.0*r*(L_1*m_1 + l_t*m_b), 1.0*I_w + 1.0*m_1*r**2 + 1.0*m_b*r**2 + 1.0*m_w*r**2]])

In [77]:
Cl

Matrix([
[0],
[0]])

In [78]:
Gl

Matrix([
[-1.0*g*(L_1*m_1 + l_t*m_b)*theta_P(t)],
[                                    0]])

In [79]:
Wl

Matrix([
[1.0, 1.0*l_t],
[ -1,     l_t]])

In [90]:
param = {I_1: 0.11, I_w:0.11, I_b:3.089539709502296,  m_b:25.278,  m_1:25.278,  m_w:25.278, r:0.069, L_1: 0.21, l_t:1.022142,  g:9.81}

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

Mlp_inv = Mlp.inv()
qdd_rhs_A = Mlp_inv*(-Clp -Glp)
qdd_rhs_B = Mlp_inv*Wlp*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

ss0 = [A, B, C, D]
sys0 = control.ss(*[pl.array(mat_i.subs(param)).astype(float) for mat_i in ss0])
mprint(sys0)

A = [[  0.           0.           1.           0.        ]
     [  0.           0.           0.           1.        ]
     [ 14.60584998   0.           0.           0.        ]
     [-66.63714437   0.           0.           0.        ]]

B = [[ 0.          0.        ]
     [ 0.          0.        ]
     [ 0.265897   -0.17406177]
     [-3.33605421  2.96407486]]

C = [[1. 0. 0. 0.]
     [0. 1. 0. 0.]
     [0. 0. 1. 0.]
     [0. 0. 0. 1.]]

D = [[0. 0.]
     [0. 0.]
     [0. 0.]
     [0. 0.]]

